In [15]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

In [16]:

from dotenv import load_dotenv
load_dotenv() 

True

In [17]:
pdf_path = Path("nodejs.pdf")

In [18]:

load = PyPDFLoader(file_path = pdf_path)
doc = load.load()


Ignoring wrong pointing object 268 0 (offset 0)
Ignoring wrong pointing object 309 0 (offset 0)


In [19]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

In [20]:


spited = text_splitter.split_documents(documents=doc)

In [23]:
store = Chroma.from_documents(
    documents=spited,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    persist_directory="qdrant_store"
)


In [24]:
retriever = store.as_retriever(
     search_type = "similarity",
        search_kwargs = {
            "k":10 
        }
)

In [25]:
retrieved_docs = retriever.invoke("What is node js")

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [38]:
system_prompt = (
    "You are a helpful assistant that answers questions about Node.js. {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [39]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    

In [ ]:
response = rag_chain.invoke({"input": "What is node js"})
print(response["answer"])

Node.js is a JavaScript runtime environment.  This means it allows you to execute JavaScript code outside of a web browser, like on a server.  It's built on Chrome's V8 JavaScript engine, which is known for its speed and performance.  Node.js uses a non-blocking, event-driven I/O model, making it efficient for handling real-time applications and applications with a lot of I/O operations like network requests.
